In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt

import cv2
import zipfile
import os
import glob
import shutil
from sklearn.utils import shuffle

import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, 
                          Dropout, Rescaling, RandomFlip, RandomRotation, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

## Loading the dataset

In [2]:
# print the list of labels in folder
folder_path = '/kaggle/input/imageclassificationdataset'
labels = os.listdir(folder_path)
labels

['Scentless Mayweed',
 'Common wheat',
 'Charlock',
 'Black-grass',
 'Sugar beet',
 'Loose Silky-bent',
 'Maize',
 'Cleavers',
 'Common Chickweed',
 'Fat Hen',
 'Small-flowered Cranesbill',
 'Shepherd’s Purse']

In [3]:
# path to Black grass images
path = '/kaggle/input/imageclassificationdataset/Black-grass'

images = os.listdir(path)
len(images)

309

In [4]:
label_count = {}

# path to images folders 
npath = '/kaggle/input/imageclassificationdataset/'

# run a for loop on labels
for label in labels:
    images_num = len(os.listdir(npath + label))
    label_count[label] = images_num
    
label_count

{'Scentless Mayweed': 607,
 'Common wheat': 253,
 'Charlock': 452,
 'Black-grass': 309,
 'Sugar beet': 463,
 'Loose Silky-bent': 762,
 'Maize': 257,
 'Cleavers': 335,
 'Common Chickweed': 713,
 'Fat Hen': 538,
 'Small-flowered Cranesbill': 576,
 'Shepherd’s Purse': 274}

In [5]:
# find the total number of  images
images_count = 0
for value in label_count.values():
    images_count += value
    
print(images_count)

5539


In [6]:
# class labels
class_names = ['Scentless Mayweed',
 'Common wheat',
 'Charlock',
 'Black-grass',
 'Sugar beet',
 'Loose Silky-bent',
 'Maize',
 'Cleavers',
 'Common Chickweed',
 'Fat Hen',
 'Small-flowered Cranesbill',
 'Shepherd’s Purse']


class_labels = {class_name:i for i, class_name in enumerate(class_names)}
print(class_labels)

IMAGE_SIZE = (120,120)

{'Scentless Mayweed': 0, 'Common wheat': 1, 'Charlock': 2, 'Black-grass': 3, 'Sugar beet': 4, 'Loose Silky-bent': 5, 'Maize': 6, 'Cleavers': 7, 'Common Chickweed': 8, 'Fat Hen': 9, 'Small-flowered Cranesbill': 10, 'Shepherd’s Purse': 11}


## Labeling the dataset

In [7]:
# labeling the dataset
labelled_data = []

images1 = []
labels1 = []


for label in labels:
    lab = class_labels[label]
    for img in os.listdir(os.path.join('/kaggle/input/imageclassificationdataset/', label)):
        image_path = os.path.join(os.path.join(npath, label), img)
        # read the image files
        img_file = cv2.imread(image_path)
        img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2RGB)
        img_file = cv2.resize(img_file, IMAGE_SIZE)
        
        images1.append(img_file)
        labels1.append(lab)

In [8]:
# convert data into np array
images1 = np.array(images1, dtype='float32')
labels1 = np.array(labels1, dtype='int32')

In [9]:
# shuffle the train nad test datasets
(images1, labels1) = shuffle(images1, labels1, random_state=45)

In [10]:
# normalize the iamges
images_norm = images1/255.0

In [11]:
# train and test split
train_images = images1[:4500]
train_labels = labels1[:4500]

test_images = images1[4500:]
test_labels = labels1[4500:]

## Building Neural Net modeling


In [12]:
num_classes = 12

# build a model using keras layers apis
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(120,120,3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

2023-02-09 13:23:58.636464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-09 13:23:58.725442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-09 13:23:58.726267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-09 13:23:58.727808: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 120, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 118, 118, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 118, 118, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 59, 59, 64)        2

In [13]:
# compile the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['acc'])

In [14]:
# y_train and y_test labels
X_train = train_images
X_test = test_images

y_train = to_categorical(train_labels, num_classes)
y_test = to_categorical(test_labels, num_classes)

In [19]:
# creating image data gen object
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )

# callbacks
early_stop = EarlyStopping(monitor="val_acc",min_delta=0, patience=5,
                           verbose=0, mode="min", baseline=None, restore_best_weights=True)

In [20]:
# fitting model to the data
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
         validation_data=datagen.flow(X_test, y_test, batch_size=8),
         steps_per_epoch=len(X_train) / 32, epochs=20,
         callbacks = [early_stop]
         )

Epoch 1/20
140/140 [==============================] - 19s 138ms/step - loss: 0.3577 - acc: 0.8762 - val_loss: 0.4032 - val_acc: 0.8768
Epoch 2/20
140/140 [==============================] - 20s 140ms/step - loss: 0.3244 - acc: 0.8887 - val_loss: 1.7493 - val_acc: 0.6266
Epoch 3/20
140/140 [==============================] - 19s 135ms/step - loss: 0.3105 - acc: 0.8944 - val_loss: 3.8529 - val_acc: 0.4937
Epoch 4/20
140/140 [==============================] - 20s 139ms/step - loss: 0.3160 - acc: 0.8924 - val_loss: 0.5389 - val_acc: 0.8499
Epoch 5/20
140/140 [==============================] - 19s 133ms/step - loss: 0.3006 - acc: 0.8993 - val_loss: 1.5128 - val_acc: 0.6574
Epoch 6/20
140/140 [==============================] - 19s 138ms/step - loss: 0.2924 - acc: 0.9020 - val_loss: 1.8758 - val_acc: 0.6564
Epoch 7/20
140/140 [==============================] - 20s 139ms/step - loss: 0.2727 - acc: 0.8984 - val_loss: 2.0872 - val_acc: 0.6208
Epoch 8/20
140/140 [==============================] - 1

In [21]:
# save the model
model.save("seeds_clf.h5")

## Inference

In [25]:
# Inference on test data
model.predict(X_test[20:22])

array([[9.4526776e-08, 7.7245273e-02, 4.3788887e-07, 2.1857393e-04,
        6.9426465e-01, 2.4278812e-05, 3.7250116e-05, 4.2054071e-06,
        3.9024330e-06, 2.2818887e-01, 1.2399363e-05, 3.8297383e-08],
       [7.8528722e-18, 8.2241072e-14, 4.4818101e-11, 7.2802415e-28,
        1.2797480e-02, 2.9625189e-28, 9.8720247e-01, 2.0203455e-22,
        1.3683995e-13, 4.5306843e-17, 4.8426535e-31, 1.7829757e-22]],
      dtype=float32)